In [1]:
data_dir = 'data'
result_dir = 'result'

# Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy

import warnings
warnings.filterwarnings("ignore")

# Read data

In [3]:
af = 'vv'
pf = '__1__'

Xval = np.load(f'{data_dir}/{af}3.v.X.{pf}.npy', allow_pickle=True)
y_val = np.load(f'{data_dir}/{af}3.v.y.npy', allow_pickle=True)

#? features names
fts_names = [line.strip() for line in open(f'{data_dir}/{af}3.fts_cols.{pf}.txt').readlines()]

In [4]:
""" split train test """
from sklearn.model_selection import train_test_split

Xtrain, Xtest, y_train, y_test = train_test_split(Xval, y_val, test_size=0.3, random_state=42)

In [5]:
Xtrain.shape

(535242, 63)

In [6]:
# for i in range(0, len(fts_names)):
#     print(i, fts_names[i])

# Test and evaluate

In [7]:
from sklearn.metrics import classification_report, confusion_matrix, PrecisionRecallDisplay, f1_score, roc_auc_score, roc_curve, auc

def evaluate(y_true, y_pred):
    #? relabel to use classification metrics
    y_pred[y_pred == 1] = 0
    y_pred[y_pred == -1] = 1

    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    print(classification_report(y_true, y_pred, digits=4))
    roc_auc = roc_auc_score(y_true, y_pred)
    print(roc_auc)

    # display = PrecisionRecallDisplay.from_predictions(y_true, y_pred)
    # _ = display.ax_.set_title("2-class Precision-Recall curve")
    return roc_auc

In [8]:
import pickle

def save_result(model, Xtr, Xt, model_name, selected_fts_names, roc_auc, expname='', config=None):
    ra = '{:.2f}'.format(round(roc_auc * 100, 2))
    # if ra <= 70: #? not good enough to even bother
    #     return
    
    #? save model
    pickle.dump(model, open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.model.pkl','wb'))

    #? save transformed X as well
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.tr.X.npy', Xtr)
    np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.t.X.npy', Xt)
    # np.save(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.data.v.X.npy', Xv)

    #? save fts
    open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.fts.txt', 'w').write('\n'.join(selected_fts_names))
    
    #? save config
    if config is not None:
        open(f'{result_dir}/{af}4.{pf}.{model_name}.{expname}.__{ra}__.config.txt', 'w').write(config)

In [9]:
from sklearn.ensemble import IsolationForest

def test_param(x_train, x_test, n_estimators, contamination):
    model = IsolationForest(n_estimators=n_estimators, max_samples=x_train.shape[0], contamination=contamination, n_jobs=10, verbose=1)

    """ train """
    model.fit(x_train)

    """ predict & evaluate on train set """
    y_train_pred = model.predict(x_train)
    #? evaluate
    print('Evaluate on train set')
    evaluate(y_train, y_train_pred)
    print()

    """ predict & evaluate on test set """
    y_test_pred = model.predict(x_test)
    #? evaluate
    print('Evaluate on test set')
    evaluate(y_test, y_test_pred)

    del x_train, x_test
    del y_test_pred, y_train_pred
    del model

## Exp 02: Run on all features

`Dur`, `sTos`, `dTos`, `Sport`, `Dport`, `TotPkts`, `TotBytes`, `SrcBytes`, `PktsPerSec`, `BytesPerSec`, `SrcBytesPerSec`, `BytesPerPkt`, `DstBytes`, `DstBytesPerSec`, `SportHex`, `DportHex`, `sTos_-1`, `sTos_0`, `sTos_1`, `sTos_2`, `sTos_3`, `dTos_-1`, `dTos_0`, `dTos_1`, `dTos_2`, `dTos_3`, `State_CON`, `State_alltcp`, `State_INT`, `State_S_`, `State_URP`, `State_ECO`, `State_RED`, `State_REQ`, `State_ECR`, `State_URH`, `State_TXD`, `State_URFIL`, `State_R_`, `State_URN`, `State_RSP`, `State_URHPRO`, `State_A_`, `State_other`, `Flag_nan`, `Flag_S`, `Flag_A`, `Flag_P`, `Flag_R`, `Flag_F`, `Proto_udp`, `Proto_tcp`, `Proto_icmp`, `Proto_rtp`, `Proto_rtcp`, `Proto_igmp`, `Proto_arp`, `Proto_other`, `Service_80`, `Service_443`, `Service_21`, `Service_22`, `Service_25`, `Service_6667`, `Service_other`

The result increase, yet running time is much higher

```
CPU times: total: 2min 13s
Wall time: 39.8 s

CPU times: total: 13.7 s
Wall time: 13.8 s

[[751564  10016]
 [  1123   1839]]
              precision    recall  f1-score   support

           0     0.9985    0.9868    0.9926    761580
           1     0.1551    0.6209    0.2482      2962

    accuracy                         0.9854    764542
   macro avg     0.5768    0.8039    0.6204    764542
weighted avg     0.9952    0.9854    0.9898    764542

0.8038563375096434
```

In [9]:
""" select features """
selected_fts_names__02 = fts_names[:]
', '.join(selected_fts_names__02)

'Dur, sTos, dTos, Sport, Dport, TotPkts, TotBytes, SrcBytes, PktsPerSec, BytesPerSec, SrcBytesPerSec, BytesPerPkt, DstBytes, DstBytesPerSec, SportHex, DportHex, sTos_-1, sTos_0, sTos_1, sTos_2, sTos_3, dTos_-1, dTos_0, dTos_1, dTos_2, dTos_3, State_CON, State_alltcp, State_INT, State_S_, State_URP, State_ECO, State_RED, State_REQ, State_ECR, State_URH, State_TXD, State_URFIL, State_R_, State_URN, State_RSP, State_URHPRO, State_A_, State_other, Flag_nan, Flag_S, Flag_A, Flag_P, Flag_R, Flag_F, Proto_udp, Proto_tcp, Proto_icmp, Proto_rtp, Proto_rtcp, Proto_igmp, Proto_arp, Proto_other, Service_80, Service_443, Service_21, Service_22, Service_25, Service_6667, Service_other'

In [10]:
X_train__02 = Xtrain[:,:]
X_test__02 = Xtest[:,:]

print(X_train__02.shape)

(535242, 65)


In [15]:
test_param(n_estimators=50, contamination=0.05)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:   10.0s remaining:   40.3s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   10.8s finished


Evaluate on train set
[[502740  24455]
 [  5740   2307]]
              precision    recall  f1-score   support

           0     0.9887    0.9536    0.9708    527195
           1     0.0862    0.2867    0.1326      8047

    accuracy                         0.9436    535242
   macro avg     0.5375    0.6202    0.5517    535242
weighted avg     0.9751    0.9436    0.9582    535242

0.6201518408422305

Evaluate on test set
[[215213  10716]
 [  2447   1014]]
              precision    recall  f1-score   support

           0     0.9888    0.9526    0.9703    225929
           1     0.0864    0.2930    0.1335      3461

    accuracy                         0.9426    229390
   macro avg     0.5376    0.6228    0.5519    229390
weighted avg     0.9751    0.9426    0.9577    229390

0.6227740388952906


In [12]:
test_param(n_estimators=50, contamination=0.006)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    9.3s remaining:   37.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:   10.0s finished


Evaluate on train set
[[524109   3086]
 [  7921    126]]
              precision    recall  f1-score   support

           0     0.9851    0.9941    0.9896    527195
           1     0.0392    0.0157    0.0224      8047

    accuracy                         0.9794    535242
   macro avg     0.5122    0.5049    0.5060    535242
weighted avg     0.9709    0.9794    0.9751    535242

0.5049021938353658

Evaluate on test set
[[224540   1389]
 [  3411     50]]
              precision    recall  f1-score   support

           0     0.9850    0.9939    0.9894    225929
           1     0.0347    0.0144    0.0204      3461

    accuracy                         0.9791    229390
   macro avg     0.5099    0.5041    0.5049    229390
weighted avg     0.9707    0.9791    0.9748    229390

0.5041493712865682


In [14]:
test_param(n_estimators=40, contamination=0.05)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    7.7s remaining:   31.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    8.3s finished


Evaluate on train set
[[503169  24026]
 [  5310   2737]]
              precision    recall  f1-score   support

           0     0.9896    0.9544    0.9717    527195
           1     0.1023    0.3401    0.1573      8047

    accuracy                         0.9452    535242
   macro avg     0.5459    0.6473    0.5645    535242
weighted avg     0.9762    0.9452    0.9594    535242

0.647276742729914

Evaluate on test set
[[215417  10512]
 [  2283   1178]]
              precision    recall  f1-score   support

           0     0.9895    0.9535    0.9712    225929
           1     0.1008    0.3404    0.1555      3461

    accuracy                         0.9442    229390
   macro avg     0.5451    0.6469    0.5633    229390
weighted avg     0.9761    0.9442    0.9589    229390

0.6469180825626465


In [16]:
test_param(n_estimators=40, contamination=0.01)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    8.0s remaining:   32.3s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    8.7s finished


Evaluate on train set
[[522183   5012]
 [  7706    341]]
              precision    recall  f1-score   support

           0     0.9855    0.9905    0.9880    527195
           1     0.0637    0.0424    0.0509      8047

    accuracy                         0.9762    535242
   macro avg     0.5246    0.5164    0.5194    535242
weighted avg     0.9716    0.9762    0.9739    535242

0.5164345610341037

Evaluate on test set
[[223714   2215]
 [  3319    142]]
              precision    recall  f1-score   support

           0     0.9854    0.9902    0.9878    225929
           1     0.0602    0.0410    0.0488      3461

    accuracy                         0.9759    229390
   macro avg     0.5228    0.5156    0.5183    229390
weighted avg     0.9714    0.9759    0.9736    229390

0.5156123197435686


In [13]:
test_param(n_estimators=30, contamination=0.05)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    5.7s remaining:   23.1s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    5.9s finished


Evaluate on train set
[[502711  24484]
 [  5769   2278]]
              precision    recall  f1-score   support

           0     0.9887    0.9536    0.9708    527195
           1     0.0851    0.2831    0.1309      8047

    accuracy                         0.9435    535242
   macro avg     0.5369    0.6183    0.5508    535242
weighted avg     0.9751    0.9435    0.9582    535242

0.618322423031074

Evaluate on test set
[[215262  10667]
 [  2458   1003]]
              precision    recall  f1-score   support

           0     0.9887    0.9528    0.9704    225929
           1     0.0859    0.2898    0.1326      3461

    accuracy                         0.9428    229390
   macro avg     0.5373    0.6213    0.5515    229390
weighted avg     0.9751    0.9428    0.9578    229390

0.6212933439548949


In [12]:
test_param(n_estimators=30, contamination=0.01)

[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    6.8s remaining:   27.6s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    7.0s finished


Evaluate on train set
[[522097   5098]
 [  7792    255]]
              precision    recall  f1-score   support

           0     0.9853    0.9903    0.9878    527195
           1     0.0476    0.0317    0.0381      8047

    accuracy                         0.9759    535242
   macro avg     0.5165    0.5110    0.5129    535242
weighted avg     0.9712    0.9759    0.9735    535242

0.5110093909734327

Evaluate on test set
[[223672   2257]
 [  3370     91]]
              precision    recall  f1-score   support

           0     0.9852    0.9900    0.9876    225929
           1     0.0388    0.0263    0.0313      3461

    accuracy                         0.9755    229390
   macro avg     0.5120    0.5082    0.5095    229390
weighted avg     0.9709    0.9755    0.9731    229390

0.5081515574177444


In [45]:
del X_train__02
del X_test__02